In [1]:
from gurobipy import *
import gurobipy as gp
import numpy as np

In [2]:
model = gp.Model("Warehouse Inventory Optimization")

Restricted license - for non-production use only - expires 2023-10-25


In [3]:
# Model names
names = [
    "iphone11", "iphone11pro", "iphone11pro_max", "iphone12", "iphone12pro", "iphone12pro_max",
    "iphone8", "iphone8plus", "iphoneX", "iphoneXS", "nova7", "P30", "P40", "S10", "S10+", "S20", "S20+",
    "S8", "S9"
]

In [4]:
# Warehouses
W = ["Calgary", "Halifax", "Montreal", "Toronto", "Vancouver"]

In [5]:
# Profit
P = [92.67, 114.81, 121.94, 101.8, 143.54, 157.05, 41.96, 53.38,
     76.63, 85.23, 30.9, 61.64, 81.16, 60.83, 85.46, 111.25, 131.79, 34.2, 53.38]

In [6]:
# Weight in grams
weight = [194, 188, 226, 162, 187, 226, 148, 202,
           174, 177, 180, 165, 175, 157, 175, 163, 186, 155, 163]

In [7]:
# Base overstocking fee
over = [5, 3.41, 3.22, 4.33, 5.5]

In [8]:
# maintenance fee (fixed)
main = [199, 130, 112, 236, 331]

In [9]:
# Regular capacity (based on actual dataset)
base = [1741, 1740, 1740, 1741, 3481]

In [10]:
# Safety range (to avoid overstocking)
L = [0.03, 0.07, 0.01, 0.04, 0.02]

In [11]:
# Maximum capacity for each warehouse
e = [2000, 2000, 2000, 2000, 3700]

In [12]:
# State of nature (based on actual dataset)
# The proportion for each model across all warehouses
S = [0.088863353, 0.020204922, 0.034855884, 0.028344345, 0.065689936, 0.043091066, 0.033706789,
     0.063391746, 0.031887389, 0.117686489, 0.032557694, 0.007660634, 0.007277602, 0.048740783,
     0.038494685, 0.035430432, 0.007277602, 0.254811836, 0.040026812]


In [13]:
m_len = len(names) # number of models
w_len = len(W) # number of warehouses(WHs)

# Number of model z in WH j
Y = model.addVars(w_len, m_len, lb=0, vtype = GRB.INTEGER, 
                  name = [n + "_" + "warehouse_" + w for w in W for n in names])

In [14]:
# Overstocking amount: optimal allocation - regular capacity * (1 + safety range)
# If there is no overstocking, the number should be 0.
overN = model.addVars(w_len, lb=0, vtype=GRB.CONTINUOUS, name = ["overstocked amount in warehouse "+w for w in W])

In [15]:
# Dynamic overstocking price: related to the overstocking amount and base overstocking fee
# But calculate the gap between optimal allocation and regular capacity instead of safety capacity
overP = model.addVars(w_len, lb=0, vtype=GRB.CONTINUOUS, name = ["overstocked price in warehouse "+w for w in W])

In [16]:
# Raw profit - transfer fee - maintenance fee - overstocking fee
model.setObjective(sum(P[z]*Y[j, z] - 0.02*weight[z]*Y[j, z] 
                       for z in range(m_len) for j in range(w_len))
                    - sum(main[j] for j in range(w_len))
                    - sum(overN[j] * overP[j] for j in range(w_len))
                   ,GRB.MAXIMIZE)

In [17]:
# Overstocking amount: optimal allocation - regular capacity * (1 + safety range)
for j in range(w_len):
    model.addConstr(overN[j] == sum(Y[j, z] for z in range(m_len)) - base[j]*(1+L[j]))

In [18]:
# Dynamic overstocking price: related to the overstocking amount and base overstocking fee
# Using different functions/strategies for different warehouses since the costs of overstocking are different among regions
# For example, the costs of overstocking are extremely high in Toronto and Vancouver since higher rents
model.addConstr(overP[0] == over[0] + (overN[0] + base[0]*L[0]) / 3)
model.addConstr(overP[3] == over[3] * (overN[3] + base[3]*L[3]))
model.addConstr(overP[2] == over[2] + (overN[2] + base[2]*L[2]) ** 2 / 10)
model.addConstr(overP[1] == over[1] + (overN[1] + base[1]*L[1]) / 10)
model.addConstr(overP[4] == over[4]*100 + (overN[4] + base[4]*L[4]))

<gurobi.Constr *Awaiting Model Update*>

In [19]:
# Setting each model z in WH j within a resonable range (0.9, 1.1) of the state of nature.
# The number of models should not deviate that much compared to state of nature.
for j in range(w_len):
    for z in range(m_len):
        model.addConstr(Y[j,z] <= base[j]*(1+L[j]) * S[z] * 1.1)
        model.addConstr(Y[j,z] >= base[j]*(1+L[j]) * S[z] * 0.9)

In [20]:
# Sum of all models in each warehouse should not exceed the maximum capicity
for j in range(w_len):
    model.addConstr(sum(Y[j, z] for z in range(m_len)) <= e[j])

In [21]:
model.Params.NonConvex = 2
model.optimize()

Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 204 rows, 105 columns and 393 nonzeros
Model fingerprint: 0xd2309e7e
Model has 5 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 10 continuous, 95 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 4e+00]
  QMatrix range    [1e-01, 1e-01]
  QLMatrix range   [1e+00, 3e+00]
  Objective range  [3e+01, 2e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 4e+03]
  QRHS range       [3e+01, 3e+01]
Presolve removed 203 rows and 84 columns
Presolve time: 0.01s
Presolved: 6 rows, 22 columns, 31 nonzeros
Presolved model has 2 bilinear constraint(s)
Variable types: 3 continuous, 19 integer (0 binary)

Root relaxation: objective 8.082384e+05, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node

In [22]:
# Result demonstration
print("Maximum profit:", round(model.objval, 3))
total = []
for v in model.getVars():
    print(v.varName, "=", round(v.x))
    total.append(round(v.x))

Maximum profit: 807054.4
iphone11_warehouse_Calgary = 175
iphone11pro_warehouse_Calgary = 39
iphone11pro_max_warehouse_Calgary = 68
iphone12_warehouse_Calgary = 55
iphone12pro_warehouse_Calgary = 129
iphone12pro_max_warehouse_Calgary = 84
iphone8_warehouse_Calgary = 55
iphone8plus_warehouse_Calgary = 109
iphoneX_warehouse_Calgary = 62
iphoneXS_warehouse_Calgary = 232
nova7_warehouse_Calgary = 53
P30_warehouse_Calgary = 15
P40_warehouse_Calgary = 14
S10_warehouse_Calgary = 96
S10+_warehouse_Calgary = 75
S20_warehouse_Calgary = 69
S20+_warehouse_Calgary = 14
S8_warehouse_Calgary = 412
S9_warehouse_Calgary = 78
iphone11_warehouse_Halifax = 181
iphone11pro_warehouse_Halifax = 41
iphone11pro_max_warehouse_Halifax = 71
iphone12_warehouse_Halifax = 58
iphone12pro_warehouse_Halifax = 134
iphone12pro_max_warehouse_Halifax = 88
iphone8_warehouse_Halifax = 69
iphone8plus_warehouse_Halifax = 129
iphoneX_warehouse_Halifax = 65
iphoneXS_warehouse_Halifax = 241
nova7_warehouse_Halifax = 55
P30_wareho

In [23]:
# Stocking for each warehouse
calgary = total[0:19]
print(sum(calgary))

1834


In [24]:
halifax = total[19:38]
print(sum(halifax))

1939


In [25]:
montreal = total[38:57]
print(sum(montreal))

1762


In [26]:
toronto = total[57:76]
print(sum(toronto))

1811


In [27]:
vancouver = total[76:95]
print(sum(vancouver))

3551


In [28]:
over_num = total[95:100]
print(over_num)

[41, 77, 5, 0, 0]


In [29]:
over_price = total[100:]
print(over_price)

[36, 23, 52, 303, 620]


In [30]:
# The optimal allocation for each model in all warehouses
sum_model = []
for i in range(m_len):
    sum_model.append(calgary[i]+halifax[i]+montreal[i]+toronto[i]+vancouver[i])
print(sum_model)

[1050, 237, 411, 333, 775, 508, 341, 646, 358, 1393, 319, 81, 84, 509, 453, 417, 84, 2481, 417]


In [31]:
# Sum of optimal allocation
print(sum(sum_model))

10897


In [32]:
# Actual data gained in our dataset, calculate the project using our objective function without considering overstocking strategy
actual_data = [928, 211, 364, 296, 686, 450, 352, 662, 333, 1229, 340,
               80, 76, 509, 402, 370, 76, 2661, 418]

actual_profit = 0
for i in range(m_len):
    actual_profit += P[i] * actual_data[i] - 0.02 * weight[i] * actual_data[i]
print(actual_profit - sum(main))

747984.02


In [33]:
# Sum of actual data allocation
print(sum(actual_data))

10443


In [34]:
# Recall the optimal profit
print("Maximum profit:", round(model.objval, 3))

Maximum profit: 807054.4


In [35]:
# Comparsion between original strategy to overstocking strategy
print("Difference:", round(model.objval - actual_profit - sum(main), 3))

Difference: 57054.38
